In [2]:
conda install keras

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py37haa95532_0         2.9 MB
    ------------------------------------------------------------
                                           Total:         2.9 MB

The following packages will be UPDATED:

  conda                                        4.8.5-py37_0 --> 4.9.2-py37haa95532_0




conda-4.9.2          | 2.9 MB    |            |   0% 
conda-4.9.2          | 2.9 MB    |            |   1% 
conda-4.9.2          | 2.9 MB    | ##1        |  22% 
conda-4.9.2          | 2.9 MB    | #####5     |  56% 
conda-4.9.2          | 2.9 MB    | ########8  |  88% 
conda-4.9.2          | 2.9 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying tran

In [1]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))


Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(150, 150, 3))) # our images are 150*150
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.15))
model.add(Dense(64, activation="relu"))
model.add(Dense(6, activation="softmax"))


In [5]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(validation_split=0.2)
train_generator = datagen.flow_from_directory("intel-images/seg_train/seg_train", batch_size=32, target_size=(150,150), class_mode="sparse", subset="training")
validation_generator = datagen.flow_from_directory("intel-images/seg_train/seg_train", batch_size=32, target_size=(150, 150), class_mode="sparse", subset="validation")
model.fit_generator(train_generator, epochs=2, validation_data=validation_generator)

# epochs set to 2 to make it go fast, but you can set it higher for a better accuracy

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'intel-images/seg_train/seg_train'

In [5]:
test_datagen = ImageDataGenerator()
test_generator = datagen.flow_from_directory("intel-images/seg_test/seg_test", target_size=(150,150), class_mode="sparse")
print(model.evaluate_generator(test_generator))

Found 3004 images belonging to 6 classes.
[1.781386375427246, 0.5692409873008728]


In [6]:
datagen = ImageDataGenerator(rotation_range=30, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)

In [9]:
model.fit_generator(train_generator, epochs=1, validation_data=validation_generator, steps_per_epoch=350, validation_steps=32)

# epochs and steps_per_epoch set very low here so you can run the notebook faster

Epoch 1/1
350/350 [==============================] - 27s 78ms/step - loss: 0.9480 - accuracy: 0.7225 - val_loss: 1.5040 - val_accuracy: 0.7207


In [10]:
model.save("images_model.h5")

In [11]:
import keras.models
model = keras.models.load_model("images_model.h5")